#### Generación de tabla "results_movie_genre_language"


##### Librerías

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
file_date = dbutils.widgets.get("file_date")

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Obtener datos (Silver)

In [0]:
df_genre = spark.read.format("delta").load(f"{silver_folder_path}/genres")
df_language = spark.read.format("delta").load(f"{silver_folder_path}/languages")
df_movie = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{file_date}'")
df_movie_language = spark.read.format("delta").load(f"{silver_folder_path}/movies_languages").filter(f"file_date = '{file_date}'")
df_movie_genre = spark.read.format("delta").load(f"{silver_folder_path}/movie_genres").filter(f"file_date = '{file_date}'")

In [0]:
df_movie_languages = df_movie_language.join(df_language,df_movie_language.language_id==df_language.language_id).select(df_movie_language.movie_id,df_language.language_name,df_language.language_id)
df_movie_genres = df_movie_genre.join(df_genre,df_movie_genre.genre_id==df_genre.genre_id).select(df_movie_genre.movie_id,df_genre.genre_name,df_genre.genre_id)

In [0]:
df_results_movie_genre_language =  df_movie.join(df_movie_languages,df_movie.movie_id==df_movie_languages.movie_id) \
                                          .join(df_movie_genres,df_movie.movie_id==df_movie_genres.movie_id) \
                                          .select(df_movie.movie_id,df_movie_languages.language_id,df_movie_genres.genre_id,df_movie.title,df_movie.duration_time,df_movie.release_date,df_movie.vote_average,df_movie_languages.language_name,df_movie_genres.genre_name) \
                                          .withColumn("created_date",lit(file_date)) \
                                          .orderBy(df_movie.release_date.desc())

In [0]:
display(df_results_movie_genre_language)

##### Escribir resultado en Gold

In [0]:
merge_condition="target.movie_id = source.movie_id  AND target.language_id = source.language_id AND target.genre_id = source.genre_id AND target.created_date = source.created_date"
merge_delta_lake(df_results_movie_genre_language,"movie_gold","results_movie_genre_language",gold_folder_path,merge_condition,"created_date")

In [0]:
%sql
SELECT * FROM movie_gold.results_movie_genre_language